In [35]:
import pandas as pd
import os

In [36]:
# Directory where CSV files are stored
csv_directory = '/Users/wickywonka/CMU/coursework/17735/project/17735-project/ExtractedData'

# Lists to hold data from each file
logon_data = []
logoff_data = []

# Iterate over each CSV file in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        # Full path to the CSV file
        csv_path = os.path.join(csv_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_path, names=['User', 'Timestamp', 'PC', 'ActivityType', 'Action'])

        # Convert Timestamp to datetime
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%m/%d/%Y %H:%M:%S')

        # Append DataFrame to the list by action
        logon_data.append(df[df['Action'] == 'Logon'])
        logoff_data.append(df[df['Action'] == 'Logoff'])

# Concatenate all logon and logoff data into two separate DataFrames
logon_df = pd.concat(logon_data)
logoff_df = pd.concat(logoff_data)

# Calculate means for logon and logoff
logon_means = logon_df.groupby('User')['Timestamp'].apply(lambda x: x.mean().time())
logoff_means = logoff_df.groupby('User')['Timestamp'].apply(lambda x: x.mean().time())

# Calculate modes for logon and logoff
logon_modes = logon_df.groupby('User')['Timestamp'].apply(lambda x: x.dt.time.mode()[0])
logoff_modes = logoff_df.groupby('User')['Timestamp'].apply(lambda x: x.dt.time.mode()[0])

# Convert Series to DataFrame for CSV output
logon_means_df = logon_means.reset_index().rename(columns={'Timestamp': 'Logon_mean_time'})
logon_modes_df = logon_modes.reset_index().rename(columns={'Timestamp': 'Logon_mode_time'})

logoff_means_df = logoff_means.reset_index().rename(columns={'Timestamp': 'Logoff_mean_time'})
logoff_modes_df = logoff_modes.reset_index().rename(columns={'Timestamp': 'Logoff_mode_time'})

# Save the DataFrames to CSV files
# logon_means_df.to_csv('logon_means.csv', index=False)
# logon_modes_df.to_csv('logon_modes.csv', index=False)
# logoff_means_df.to_csv('logoff_means.csv', index=False)
# logoff_modes_df.to_csv('logoff_modes.csv', index=False)

# These variables hold the respective statistics for later use in your code:
# logon_means_df, logon_modes_df, logoff_means_df, logoff_modes_df

In [37]:
logon_means_df

,User,Logon_mean_time
0,DTAA/AAA0371,02:49:05.759328
1,DTAA/AAC0344,14:25:33.652941
2,DTAA/AAC0599,14:25:19.326470
3,DTAA/AAH0734,14:55:31.823529
4,DTAA/AAK0658,14:54:54.188235
...,...,...
995,DTAA/ZGH0528,22:42:44.836833
996,DTAA/ZKE0662,00:04:52.294308
997,DTAA/ZKH0388,06:50:55.272590
998,DTAA/ZKN0548,14:55:06.173529


In [38]:
logon_modes_df.head()

,User,Logon_mode_time
0,DTAA/AAA0371,07:54:21
1,DTAA/AAC0344,07:51:03
2,DTAA/AAC0599,07:56:28
3,DTAA/AAH0734,08:20:42
4,DTAA/AAK0658,08:22:33


In [39]:
logoff_means_df.head()

,User,Logoff_mean_time
0,DTAA/AAA0371,21:04:05.442996
1,DTAA/AAC0344,22:27:35.076470
2,DTAA/AAC0599,22:27:32.920588
3,DTAA/AAH0734,23:58:02.238235
4,DTAA/AAK0658,22:58:07.538235


In [40]:
logoff_modes_df.head()

,User,Logoff_mode_time
0,DTAA/AAA0371,14:44:56
1,DTAA/AAC0344,15:55:10
2,DTAA/AAC0599,15:56:17
3,DTAA/AAH0734,17:34:26
4,DTAA/AAK0658,16:38:32


In [41]:
# Lists to hold data from each file
connect_data = []
disconnect_data = []

# Iterate over each CSV file in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        # Full path to the CSV file
        csv_path = os.path.join(csv_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_path, names=['User', 'Timestamp', 'PC', 'ActivityType', 'Action'])

        # Convert Timestamp to datetime
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%m/%d/%Y %H:%M:%S')

        # Filter DataFrame by action and append to the respective list
        connect_data.append(df[df['Action'] == 'Connect'])
        disconnect_data.append(df[df['Action'] == 'Disconnect'])

# Concatenate all connect and disconnect data into two separate DataFrames
connect_df = pd.concat(connect_data)
disconnect_df = pd.concat(disconnect_data)

# Calculate means for connect and disconnect
connect_means = connect_df.groupby('User')['Timestamp'].apply(lambda x: x.mean().time())
disconnect_means = disconnect_df.groupby('User')['Timestamp'].apply(lambda x: x.mean().time())

# Calculate modes for connect and disconnect
# We use scipy's mode function which returns the first mode in case of multimodal data
connect_modes = connect_df.groupby('User')['Timestamp'].apply(lambda x: x.dt.time.mode()[0])
disconnect_modes = disconnect_df.groupby('User')['Timestamp'].apply(lambda x: x.dt.time.mode()[0])

# Convert Series to DataFrame for CSV output
connect_means_df = connect_means.reset_index().rename(columns={'Timestamp': 'Connect_mean_time'})
connect_modes_df = connect_modes.reset_index().rename(columns={'Timestamp': 'Connect_mode_time'})

disconnect_means_df = disconnect_means.reset_index().rename(columns={'Timestamp': 'Disconnect_mean_time'})
disconnect_modes_df = disconnect_modes.reset_index().rename(columns={'Timestamp': 'Disconnect_mode_time'})

# Save the DataFrames to CSV files
# connect_means_df.to_csv('connect_means.csv', index=False)
# connect_modes_df.to_csv('connect_modes.csv', index=False)
# disconnect_means_df.to_csv('disconnect_means.csv', index=False)
# disconnect_modes_df.to_csv('disconnect_modes.csv', index=False)

# These variables hold the respective statistics for later use in your code:
# connect_means_df, connect_modes_df, disconnect_means_df, disconnect_modes_df

In [42]:
connect_means_df.head()

,User,Connect_mean_time
0,DTAA/ABB0272,19:20:46.508823
1,DTAA/ABS0726,00:06:48.806451
2,DTAA/ACH0803,14:59:06.423791
3,DTAA/AFF0760,03:13:46.333333
4,DTAA/AFH0331,10:56:46.738007


In [43]:
connect_modes_df.head()

,User,Connect_mode_time
0,DTAA/ABB0272,15:47:45
1,DTAA/ABS0726,01:01:07
2,DTAA/ACH0803,16:05:54
3,DTAA/AFF0760,00:57:03
4,DTAA/AFH0331,07:57:22


In [44]:
disconnect_means_df.head()

,User,Disconnect_mean_time
0,DTAA/ABB0272,22:43:29.416393
1,DTAA/ABS0726,00:36:46.290322
2,DTAA/ACH0803,06:03:02.383333
3,DTAA/AFF0760,03:44:57.333333
4,DTAA/AFH0331,09:26:35.647541


In [45]:
disconnect_modes_df.head()

,User,Disconnect_mode_time
0,DTAA/ABB0272,14:35:34
1,DTAA/ABS0726,01:01:46
2,DTAA/ACH0803,16:43:48
3,DTAA/AFF0760,01:02:11
4,DTAA/AFH0331,17:47:54


In [ ]:
df_all = pd.DataFrame()

# Read all CSV files and concatenate into a single DataFrame
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_directory, filename)
        df_temp = pd.read_csv(file_path, header=None)  # Assuming the CSV has no header
        df_temp.columns = ['User', 'Timestamp', 'PC', 'Activity', 'Action']  # Assign column names
        df_all = pd.concat([df_all, df_temp])

# Ensure 'Timestamp' is a datetime object
df_all['Timestamp'] = pd.to_datetime(df_all['Timestamp'], format='%m/%d/%Y %H:%M:%S')

# Filter out 'Logon' and 'Logoff' activities into separate DataFrames
df_logon = df_all[df_all['Action'] == 'Logon']
df_logoff = df_all[df_all['Action'] == 'Logoff']

# Define a function to perform the aggregation and transformation
def aggregate_user_pc_log_activities(df_activity):
    # Aggregate data to count the total number of activities per user per PC
    df_user_pc = df_activity.groupby(['User', 'PC']).size().reset_index(name='pc_activities_per_user_total')

    # Calculate the unique count of PCs per user
    df_user_pc['unique_pc_count'] = df_user_pc.groupby('User')['PC'].transform('nunique')

    # Keep only the 'User' and 'unique_pc_count' columns and drop duplicates
    df_user_pc = df_user_pc[['User', 'unique_pc_count']].drop_duplicates()

    return df_user_pc

# Apply the function to both logon and logoff DataFrames
df_user_pc_logon = aggregate_user_pc_log_activities(df_logon)
df_user_pc_logoff = aggregate_user_pc_log_activities(df_logoff)

# Optionally, save these dataframes to CSV
# df_user_pc_logon.to_csv('user_pc_logon.csv', index=False)
# df_user_pc_logoff.to_csv('user_pc_logoff.csv', index=False)

In [34]:
df_user_pc_logon

,User,unique_pc_count
0,DTAA/AAA0371,16
16,DTAA/AAC0344,1
17,DTAA/AAC0599,1
18,DTAA/AAH0734,1
19,DTAA/AAK0658,1
...,...,...
22244,DTAA/ZGH0528,1
22245,DTAA/ZKE0662,1
22246,DTAA/ZKH0388,44
22290,DTAA/ZKN0548,1
